<a href="https://colab.research.google.com/github/manuaishika/mu/blob/data/cmu_stn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm

# Data loading with error handling
try:
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
except Exception as e:
    print(f"Error loading data: {e}")
    exit(1)

# Network definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )
        self.fc_loc = nn.Sequential(
            nn.Linear(10 * 3 * 3, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 10 * 3 * 3)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)
        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x

    def forward(self, x):
        x = self.stn(x)
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Device setup with fallback
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Training with progress bar and validation
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

def train(epoch):
    model.train()
    total_loss = 0
    with tqdm(train_loader, desc=f"Epoch {epoch}", unit="batch") as t:
        for batch_idx, (data, target) in enumerate(t):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            t.set_postfix(loss=loss.item())
            if batch_idx % 100 == 0 and batch_idx > 0:
                print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                      f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    return total_loss / len(train_loader)

def validate():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            total_loss += F.nll_loss(output, target).item()
    return total_loss / len(test_loader)

for epoch in range(1, 11):
    train_loss = train(epoch)
    val_loss = validate()
    print(f'Epoch {epoch} completed - Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}')

# Visualization with saved output
def visualize_stn():
    model.eval()
    with torch.no_grad():
        data, _ = next(iter(test_loader))
        data = data.to(device)
        transformed_data = model.stn(data)
        data = data.cpu().numpy()
        transformed_data = transformed_data.cpu().numpy()
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
        ax1.imshow(data[0, 0], cmap='gray')
        ax1.set_title('Original')
        ax1.set_xlabel('X')
        ax1.set_ylabel('Y')
        ax2.imshow(transformed_data[0, 0], cmap='gray')
        ax2.set_title('Transformed')
        ax2.set_xlabel('X')
        ax2.set_ylabel('Y')
        plt.savefig('stn_visualization.png')
        plt.close()

visualize_stn()
print("Visualization saved as 'stn_visualization.png'")

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.47MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 161kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.51MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.11MB/s]


Using device: cpu


Epoch 1:   0%|          | 0/938 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5082: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5015: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
Epoch 1:  11%|█         | 102/938 [00:05<00:55, 15.12batch/s, loss=2.18]

Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.242711


Epoch 1:  22%|██▏       | 204/938 [00:12<00:39, 18.60batch/s, loss=1.76]

...

Train Epoch: 10 [51200/60000 (85%)]	Loss: 0.142780


Epoch 10:  96%|█████████▋| 904/938 [00:51<00:01, 18.83batch/s, loss=0.161]

Train Epoch: 10 [57600/60000 (96%)]	Loss: 0.129389


Epoch 10: 100%|██████████| 938/938 [00:54<00:00, 17.18batch/s, loss=0.823]


Epoch 10 completed - Train Loss: 0.139035, Validation Loss: 0.086410
Visualization saved as 'stn_visualization.png'
